In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Border, Side, PatternFill, Font
from config_parameters import ANALYSIS_TICKER_INTERVAL_FOLDER,SUMMARY_SIMULATIONS_FILE
import os

# Define cell styling
green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")

thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# Optional: draw border around a vertical block of cells
def draw_border(ws, start_row, end_row, col):
    for row in range(start_row, end_row + 1):
        cell = ws.cell(row=row, column=col)
        cell.border = Border(
            top=thin_border if row == start_row else None,
            bottom=thin_border if row == end_row else None,
            left=thin_border,
            right=thin_border
        )

# Optional: draw border around a merged row
def draw_merged_row_border(ws, row, start_col, end_col):
    for col in range(start_col, end_col + 1):
        cell = ws.cell(row=row, column=col)
        cell.border = Border(
            top=thin_border,
            bottom=thin_border,
            left=thin_border if col == start_col else None,
            right=thin_border if col == end_col else None
        )

# Input time range


def generate_analyse_for_tickers(start_date,end_date, intervals):
    # Load simulation summary file
    summary_file_path = os.path.join(SUMMARY_SIMULATIONS_FILE, f"{start_date}_to_{end_date}.xlsx")
    summary_df = pd.read_excel(summary_file_path)

    # Prepare workbook for output
    wb = Workbook()
    wb.remove(wb.active)

    # Extract unique tickers and strategies
    tickers = summary_df['Ticker'].unique()
    summary_df['Strategy_ID'] = summary_df['Strategy'].astype(str) + ' - ' + summary_df['Description']
    strategies = sorted(summary_df['Strategy_ID'].unique(), key=lambda x: float(x.split(' - ')[0]))
    intervals = summary_df['Interval'].unique()

    # Setup output directory and file
    analysis_dir = ANALYSIS_TICKER_INTERVAL_FOLDER
    os.makedirs(analysis_dir, exist_ok=True)
    output_file = os.path.join(analysis_dir, f"{start_date}_to_{end_date}.xlsx")
    
    # Sheet per ticker
    for ticker in tickers:
        ws = wb.create_sheet(title=str(ticker))

        # Header information
        filtered = summary_df[(summary_df['Ticker'] == ticker) & (summary_df['Interval'] == "1d")]
        if not filtered.empty:
            start_date_value = filtered.iloc[0]['Start_date']
            end_date_value = filtered.iloc[0]['End_date']
            length_days = filtered.iloc[0]['length(d)']
            deposit = filtered.iloc[0]['Deposit']
        else:
            start_date_value = "None"
            end_date_value = "None"
            length_days = "None"
            deposit = "None"


        # Main title
        ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=3+len(intervals))
        cell = ws.cell(row=1, column=1)
        cell.value = f"Analysis for ticker: {ticker}        Period: {start_date_value} - {end_date_value}    Length: {length_days}(d) Deposit: {deposit}"
        cell.alignment = Alignment(horizontal='center', vertical='center')
        cell.border = thin_border

        # Interval header row
        ws.cell(row=2, column=2).value = "Interval"
        ws.cell(row=2, column=2).border = thin_border
        for i, interval in enumerate(intervals):
            cell = ws.cell(row=2, column=3 + i)
            cell.value = interval
            cell.border = thin_border
        ws.cell(row=2, column=3 + len(intervals)).value = "Average"
        ws.cell(row=2, column=3 + len(intervals)).border = thin_border

        current_row = 3

        # Strategy rows
        for strategy_id in strategies:
            strat_df = summary_df[(summary_df['Ticker'] == ticker) & (summary_df['Strategy_ID'] == strategy_id)]
            if strat_df.empty:
                continue

            # Merge and write strategy title
            ws.merge_cells(start_row=current_row, start_column=1, end_row=current_row+3, end_column=1)
            cell = ws.cell(row=current_row, column=1)
            cell.value = f"Strategy {strategy_id}"
            cell.alignment = Alignment(wrap_text=True, vertical='center')
            cell.border = thin_border

            # Row: Number of trades
            ws.cell(row=current_row, column=2).value = "No. Trades"
            ws.cell(row=current_row, column=2).border = thin_border
            values = []
            for i, interval in enumerate(intervals):
                v = strat_df[strat_df['Interval'] == interval]['Number of Trades'].values
                v = v[0] if len(v) else ''
                cell = ws.cell(row=current_row, column=3 + i)
                cell.value = v
                cell.border = thin_border
                if isinstance(v, (int, float)):
                    values.append(v)
            avg = round(sum(values) / len(values), 2) if values else ''
            ws.cell(row=current_row, column=3 + len(intervals)).value = avg
            ws.cell(row=current_row, column=3 + len(intervals)).border = thin_border
            current_row += 1

            # Row: Profit Factor
            ws.cell(row=current_row, column=2).value = "Profit Factor"
            ws.cell(row=current_row, column=2).border = thin_border
            values = []
            for i, interval in enumerate(intervals):
                v = strat_df[strat_df['Interval'] == interval]['Profit Factor'].values
                v = v[0] if len(v) else ''
                cell = ws.cell(row=current_row, column=3 + i)
                cell.value = round(v, 2) if isinstance(v, (int, float)) else v
                cell.number_format = '0.00'
                cell.border = thin_border
                if isinstance(v, (int, float)):
                    values.append(v)
            avg = round(sum(values) / len(values), 2) if values else ''
            cell = ws.cell(row=current_row, column=3 + len(intervals))
            cell.value = avg
            cell.number_format = '0.00'
            cell.border = thin_border
            current_row += 1

            # Row: P&L after fees
            ws.cell(row=current_row, column=2).value = "P&L after fees"
            ws.cell(row=current_row, column=2).border = thin_border
            values = []
            for i, interval in enumerate(intervals):
                v = strat_df[strat_df['Interval'] == interval]['P&L after fees'].values
                v = v[0] if len(v) else ''
                cell = ws.cell(row=current_row, column=3 + i)
                cell.value = round(v) if isinstance(v, (int, float)) else v
                cell.number_format = '#,##0'
                cell.border = thin_border
                if isinstance(v, (int, float)) and v > 0:
                    cell.fill = green_fill
                if isinstance(v, (int, float)):
                    values.append(v)
            avg = round(sum(values) / len(values)) if values else ''
            cell = ws.cell(row=current_row, column=3 + len(intervals))
            cell.value = avg
            cell.number_format = '#,##0'
            if isinstance(avg, (int, float)) and avg > 0:
                cell.fill = green_fill
            cell.border = thin_border
            current_row += 1

            # Row: Max Drawdown
            ws.cell(row=current_row, column=2).value = "Max Drawdown"
            ws.cell(row=current_row, column=2).border = thin_border
            values = []
            for i, interval in enumerate(intervals):
                v = strat_df[strat_df['Interval'] == interval]['Max Drawdown'].values
                v = v[0] if len(v) else ''
                cell = ws.cell(row=current_row, column=3 + i)
                cell.value = v
                cell.border = thin_border
                if isinstance(v, (int, float)):
                    values.append(v)
            avg = round(sum(values) / len(values), 2) if values else ''
            cell = ws.cell(row=current_row, column=3 + len(intervals))
            cell.value = avg
            cell.border = thin_border
            ws.column_dimensions['A'].width = 28
            current_row += 1

    # Save final Excel file
    print(f"Saving analysis to {output_file}")
    wb.save(output_file)



start_date = "2010-01-01"
end_date = "2025-03-01"
period_id = '2'
intervals = ['1w', '3d','2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']

generate_analyse_for_tickers(start_date, end_date, intervals)

Saving analysis to ticker interval analysis\2010-01-01_to_2025-03-01.xlsx
